In [ ]:
import gymnasium as gym
import math
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/phi-2",
    torch_dtype=torch.float32 if device == "cpu" else "auto",
    trust_remote_code = True
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    torch_dtype=torch.float32 if device == "cpu" else "auto",
    # device_map="auto",
    trust_remote_code = True
)

In [ ]:
env = gym.make('CartPole-v1', render_mode="human")

for i_episode in range(10):
    observation = env.reset()
    rad = observation[0][2]
    rewards = 0
    for t in range(250):
        env.render()

        #action = env.action_space.sample()
        
        degrees = round(math.degrees(rad), 2)
        direction = "right"
        if degrees < 0:
            direction = "left"
            degrees = degrees * -1

        prompt = """Instruct: In which direction the cart of a cart pole should be moved if the pole direction is """ + str(degrees) + """ degrees off to the """ + direction + """ to keep the pole up? Only answer are 'left' or 'right'. Output:"""
        
        with torch.no_grad():
          token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
          output_ids = model.generate(
              token_ids.to(model.device),
              max_new_tokens=20,
              do_sample=True,
              temperature = 0.3,
              pad_token_id=tokenizer.eos_token_id
          )
        
        output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
        
        action = 0
        if "right" in output.lower():
            action = 1
        print(str(degrees) + " " + direction + " action: " + str(action))
        observation, reward, terminated, truncated, info = env.step(action)
        rad = observation[2]
        rewards += reward

        if terminated:
            print('Episode finished after {} timesteps, total rewards {}'.format(t+1, rewards))
            break

env.close()

: 